# Search for an Artist

**Objective:** Find an artist to download, marking the ones to skip.


In [1]:
import os, sys, json, random
import pandas as PD
import numpy as NP

In [2]:
MODULE_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(MODULE_PATH, 'data')
LOGIC_PATH = os.path.join(MODULE_PATH, 'logic')
sys.path.append(LOGIC_PATH)
print(MODULE_PATH)

C:\Documents\Code\OpenStack\Current\MediaManager\codebase\pictures_manager


In [3]:
import sqlalchemy as SQL
import database_helper as DB
print([x for x in dir(DB) if '_' not in x and len(x) > 8])
dbPath = os.path.join(DATA_PATH, 'fiction.db')
meta, engine = DB.openConnection(dbPath)
artistTb = meta.tables.get('ArtistTracker')

['openConnection']
sqlite:///C:\Documents\Code\OpenStack\Current\MediaManager\codebase\pictures_manager\data\fiction.db


In [4]:
import deviantart_scraper as SC
print([x for x in dir(SC) if '_' not in x and len(x) > 8])

['DeviantArtScraper']


In [5]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import IPython.display as DS 
from ipywidgets import IntProgress

### Search for Artist

In [6]:
# get a random artist whose favorites haven't been checked

statement = SQL.select(artistTb).where(
        (artistTb.c.favourites==False) & (artistTb.c.status!='skip') & (artistTb.c.artistType=='cosplay'))
results = statement.execute()
resultsDx = [dict(x) for x in results]

random.seed(10)
artistDx = random.choice(resultsDx)
PD.DataFrame(artistDx, index=[0]).head()

,id,collectionId,deviantId,artistType,status,favourites,collected,downloaded
0,312,angela.bz,angelabermudez,cosplay,2020-12,False,None,-1


In [27]:
# update the current artist

statement = SQL.update(artistTb
        ).where(artistTb.c.deviantId=='LadyGiselle'  #artistTb.c.collectionId=='chinhy.sou'
        ).values({ 'deviantId': 'LadyGiselle', 'favourites': True, 'downloaded': 140 })  # 
results = statement.execute()
print(statement)
print(f"rows updated: {results.rowcount:,}")

UPDATE "ArtistTracker" SET "deviantId"=?, favourites=?, downloaded=? WHERE "ArtistTracker"."deviantId" = ?
rows updated: 1


In [28]:
# check an artist

searchDeviantId = 'LadyGiselle'

statement = SQL.select(artistTb).where(SQL.or_(
    SQL.func.lower(artistTb.c.deviantId)==searchDeviantId.lower(),
    artistTb.c.collectionId.ilike(f"%{searchDeviantId.lower()}%")))
results = statement.execute()
resultsDx = [dict(x) for x in results]
PD.DataFrame(resultsDx).head()

,id,collectionId,deviantId,artistType,status,favourites,collected,downloaded
0,404,ld.giselle,LadyGiselle,cosplay,2021-05,True,None,140


In [25]:
# create new artist for skipping

deviantId = 'NitroShiro'

try:
    SQL.insert(artistTb).values(
        {'deviantId': deviantId, 'artistType': 'cosplay', 'status': 'skip', }).execute() # photographer
    #{'collectionId': 'moon.fox', 'deviantId': deviantId, 'artistType': 'cosplay', 'status': '2020-12', 'downloaded': '184'}).execute()
except Exception as ex:
    print(ex)

statement = SQL.select(artistTb).where(artistTb.c.deviantId==deviantId)
results = statement.execute()
resultsDx = [dict(x) for x in results]
PD.DataFrame(resultsDx).head()

,id,collectionId,deviantId,artistType,status,favourites,collected,downloaded
0,561,no-id,NitroShiro,cosplay,skip,False,None,None


In [15]:

SQL.delete(artistTb).where(artistTb.c.deviantId=='TsukiAnni').execute()

In [ ]:

statement = SQL.update(artistTb
        ).where(artistTb.c.deviantId=='cherryb-cosplay'
        ).values({'deviantId': 'CherryB-Cosplay', 'downloaded': '108'})  # 
results = statement.execute()
print(statement)
print(f"rows updated: {results.rowcount:,}")

In [ ]:
try:
    SQL.insert(artistTb).values(
        {'collectionId': 'nemu_', 'deviantId': 'Nemu013', 'artistType': 'cosplay', 
         'status': '2021-05', 'downloaded': 294}).execute()
except Exception as ex:
    print(ex)

statement = SQL.select(artistTb).where(artistTb.c.deviantId=='Nemu013')
results = statement.execute()
resultsDx = [dict(x) for x in results]
PD.DataFrame(resultsDx).head()